## Study of Borrowers' Reliability

### 1. Data review
Let's open the dataset and examine the general information about the data.

In [2]:
import pandas as pd

try:
    data = pd.read_csv('D:\docs\DS\data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### 2. Data preprocessing
#### 2.1. Removing missing values
Let's output the number of missing values for each column.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

The `total_income` column contains many missing values. We cannot remove 10% of the dataset's observations, nor can we leave the missing values unprocessed, as this is one of the key indicators. Since income amount is most influenced by the type of employment, we will fill in the missing values with the median value for each employment type from the income_type column. For example, if a client has an employment type of `employee` and a missing value in the `total_income` column, it will be filled with the median income among all records with the same employment type.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()
    
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

We will fill in the missing values in the `days_employed` column with the median values for each employment type in the income_type column:

In [14]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### 2.2. Handling anomalous values
Now let's look at the distribution of values in the `days_employed` column.

In [7]:
print(f"Days employed minimum: {data['days_employed'].min()} \n"
      f"Days employed maximum: {data['days_employed'].max()}"     
     )

Days employed minimum: -18388.949900568383 
Days employed maximum: 401755.40047533


In [8]:
data['days_employed'] = data['days_employed'].abs()

print(f"Days employed minimum: {data['days_employed'].min()} \n"
      f"Days employed maximum: {data['days_employed'].max()}"     
     )

Days employed minimum: 24.14163324048118 
Days employed maximum: 401755.40047533


For each employment type, we will find the median value of work experience (column `days_employed`).

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Two types (unemployed-безработный and retirees-пенсионер) have abnormally large values. Given the available dataset, handling such values is difficult, so we will leave this column unchanged, especially since the information in it is not critical for our study.  
Now let's check the situation with the number of children (column `children`)

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [12]:
data.query('children == 20 or children == -1')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
705,-1,902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
742,-1,3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
...,...,...,...,...,...,...,...,...,...,...,...,...
21140,-1,1422.668059,44,среднее,1,женат / замужем,0,F,компаньон,0,169562.091999,операции со своей недвижимостью
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,172357.950966,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


For 123 clients, the number of children is incorrect, which accounts for 0.57% of the sample. It is better to remove such observations rather than try to fix them.

In [13]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

We will convert the data type in the total_income column from float to integer using the `astype()` method.

In [15]:
data['total_income'] = data['total_income'].astype(int)

#### 2.3. Handling Duplicates
In the `education` column, there are identical values recorded differently due to uppercase and lowercase letters. We will handle these implicit duplicates by converting all values to lowercase. After that, we will check the other columns for duplicates.

In [16]:
data['education'] = data['education'].str.lower()
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()
data.shape

(21331, 12)

#### 2.4. Data Categorization
Based on the ranges below, we will create a new column `total_income_category` in the dataset with the following categories:

* 0–30000 &mdash; `'E'`
* 30001–50000 &mdash; `'D'`
* 50001–200000 &mdash; `'C'`
* 200001–1000000 &mdash; `'B'`
* 1000001 and above &mdash; `'A'`

For example, a borrower with an income of 25000 will be assigned category `'E'`, while a client earning 235000 will fall into category `'B'`.

To implement this, we will write a function and apply it using the `apply()` method.

In [21]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass
    
data['total_income_category'] = data['total_income'].apply(categorize_income)
data.shape

(21331, 13)

Let's look at the list of unique loan purposes from the `purpose` column.

In [22]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

We will write a function that, based on the data in the `purpose` column, will create a new column called `purpose_category` with the following categories:

* `'операции с автомобилем'` (car-related operations)
* `'операции с недвижимостью'` (real estate operations)
* `'проведение свадьбы'` (wedding expenses)
* `'получение образования'` (education)

For example, if the purpose column contains the substring `"на покупку автомобиля"` (for car purchase), the purpose_category column will be assigned `"операции с автомобилем"` (car-related operations).

To achieve this, we will define a function and apply it using the `apply()` method.

In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.shape

(21331, 14)

### 3. Data Analysis
#### 3.1. Number of Children vs. On-Time Loan Repayment
We will create a pivot table for the children category, displaying:
* The total number of clients in each subcategory
* The number of defaulters in each subcategory

In [24]:
children_pivot_data = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum'])
children_pivot_data

,count,sum
,debt,debt
children,,
0,14091,1063
1,4808,444
2,2052,194
3,330,27
4,41,4
5,9,0


We will calculate the share of defaulters within each group and add a new column to the dataset.

In [25]:
children_pivot_data['debtors_percent'] = children_pivot_data['sum'] / children_pivot_data['count'] * 100

# For better readability, we will round the percentage of defaulters to two decimal places.
children_pivot_data['debtors_percent'] = children_pivot_data['debtors_percent'].round(decimals=2)

# We will assign more descriptive names to the columns and display the dataset on the screen.
children_pivot_data.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
children_pivot_data

,total_amount,clients_with_debt,debtors_percent
children,,,
0,14091,1063,7.54
1,4808,444,9.23
2,2052,194,9.45
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


Let's check if there are common characteristics between defaulters without children and those with two children. We will compare the median income level and marital status of defaulters in these categories.

In [27]:
# We will create a pivot table by selecting defaulters without children and those with two children from the main dataset.

compare_children = data[(data['children'] == 2) & (data['debt'] == 1) | (data['children'] == 0) & (data['debt'] == 1)].pivot_table(
    index=['children', 'total_income_category', 'family_status'],
    values=['total_income', 'debt'],
    aggfunc=['median', 'count']).reset_index()
                                    
compare_children

children total_income_category          family_status median               \
                                                           debt total_income   
0         0                     A       гражданский брак    1.0    1030899.0   
1         0                     B  Не женат / не замужем    1.0     254793.5   
2         0                     B              в разводе    1.0     268282.0   
3         0                     B         вдовец / вдова    1.0     232026.0   
4         0                     B       гражданский брак    1.0     241915.0   
5         0                     B        женат / замужем    1.0     252996.5   
6         0                     C  Не женат / не замужем    1.0     135487.5   
7         0                     C              в разводе    1.0     131091.0   
8         0                     C         вдовец / вдова    1.0     123926.5   
9         0                     C       гражданский брак    1.0     124813.0   
10        0                     C        женат / замужем    1.0     127804.0   
11        0                     D  Не женат / не замужем    1.0      39480.5   
12        0                     D       гражданский брак    1.0      35249.0   
13        0                     D        женат / замужем    1.0      42915.0   
14        0                     E        женат / замужем    1.0      24910.5   
15        2                     B  Не женат / не замужем    1.0     225702.0   
16        2                     B              в разводе    1.0     218411.0   
17        2                     B         вдовец / вдова    1.0     320526.0   
18        2                     B       гражданский брак    1.0     341738.5   
19        2                     B        женат / замужем    1.0     290383.5   
20        2                     C  Не женат / не замужем    1.0     152645.5   
21        2                     C              в разводе    1.0      98190.5   
22        2                     C         вдовец / вдова    1.0     157362.5   
23        2                     C       гражданский брак    1.0     113218.0   
24        2                     C        женат / замужем    1.0     131068.0   
25        2                     D              в разводе    1.0      45402.0   
26        2                     D        женат / замужем    1.0      46637.0   

   count               
    debt total_income  
0      1            1  
1     50           50  
2     12           12  
3      7            7  
4     39           39  
5    112          112  
6    158          158  
7     43           43  
8     46           46  
9    186          186  
10   395          395  
11     2            2  
12     3            3  
13     7            7  
14     2            2  
15     1            1  
16     1            1  
17     1            1  
18     6            6  
19    32           32  
20     8            8  
21     4            4  
22     2            2  
23    24           24  
24   111          111  
25     2            2  
26     2            2

In [28]:
compare_children.columns = ['children', 'total_income_category', 'family_status', 
                            'median_debt', 'median_total_income', 'debt_count', 'income_count']

compare_children.drop(['median_debt', 'income_count'], axis=1, inplace=True)

compare_children

,children,total_income_category,family_status,median_total_income,debt_count
0,0,A,гражданский брак,1030899.0,1
1,0,B,Не женат / не замужем,254793.5,50
2,0,B,в разводе,268282.0,12
3,0,B,вдовец / вдова,232026.0,7
4,0,B,гражданский брак,241915.0,39
5,0,B,женат / замужем,252996.5,112
6,0,C,Не женат / не замужем,135487.5,158
7,0,C,в разводе,131091.0,43
8,0,C,вдовец / вдова,123926.5,46
9,0,C,гражданский брак,124813.0,186


In [29]:
# add the within-group percentage of defaulters for each borrower category (without children and with two children):
# divide the values from the debt_count column by the total number of defaulters: 
# 1,063 for those without children and 444 for those with two children. 
# This will give us the within-group percentage for each category.

compare_children['ingroup_percent'] = compare_children.loc[compare_children['children'] == 0, 'debt_count'] / 1063 * 100
compare_children.loc[compare_children['children'] == 2,
                     'ingroup_percent'] = compare_children.loc[compare_children['children'] == 2, 'debt_count'] / 444 * 100

compare_children['ingroup_percent'] = compare_children['ingroup_percent'].round(decimals=2)

compare_children.sort_values(['children', 'debt_count'], ascending=(True, False))

,children,total_income_category,family_status,median_total_income,debt_count,ingroup_percent
10,0,C,женат / замужем,127804.0,395,37.16
9,0,C,гражданский брак,124813.0,186,17.50
6,0,C,Не женат / не замужем,135487.5,158,14.86
5,0,B,женат / замужем,252996.5,112,10.54
1,0,B,Не женат / не замужем,254793.5,50,4.70
8,0,C,вдовец / вдова,123926.5,46,4.33
7,0,C,в разводе,131091.0,43,4.05
4,0,B,гражданский брак,241915.0,39,3.67
2,0,B,в разводе,268282.0,12,1.13
3,0,B,вдовец / вдова,232026.0,7,0.66


37% of defaulters among borrowers without children and 25% among those with two children share approximately the same characteristics in other categories:

* Similar median income for category C borrowers (127,804 and 131,068 RUB, respectively).
* Same marital status: married.

Indeed, all else being equal, borrower reliability tends to decrease as the number of children in the family increases.

**Conclusion**:  
Clients without children are less likely to have loan debt than families with children. The more children a borrower has, the higher the likelihood of payment delays:

* 7.54% of defaulters among clients without children.
* 9.23% of defaulters among clients with one child.
* 9.45% of defaulters among clients with two children.

There are not enough clients with three or more children to form a representative subgroup.

#### 3.2. Marital Status vs. On-Time Loan Repayment
First, let's examine the different types of marital status in the dataset and how frequently each occurs.

In [30]:
print('Семейное положение', data['family_status'].value_counts(), sep='\n')

Семейное положение
family_status
женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: count, dtype: int64


We will conduct additional categorization before creating a pivot table:

* Since having children significantly affects the ability to repay a loan, we assume that single parents are more likely to become defaulters than single individuals without children. We will create two separate groups of single borrowers: with children and without children.
* However, we will not combine clients in official marriages and civil unions—this will allow us to analyze whether the lack of legal obligations affects clients' financial reliability.

Thus, we define four categories:
1. `в браке` (Married) – People in a legally registered marriage.
2. `не в браке` (Not married) – People cohabiting without official marriage.
3. `один, без детей` (Single, no children) – Clients without a partner and without children (including divorced and widowed individuals).
4. `один, с детьми` (Single, with children) – Clients with children who do not have a partner (including divorced and widowed individuals).

Now, let's write a function to assign these categories and add them to the dataset.

In [31]:
def married_or_not (row):
    status = row['family_status']
    children = row['children']
    
    if status == 'женат / замужем':
        return 'в браке'
    if status == 'гражданский брак':
        return 'не в браке'
    if status == 'Не женат / не замужем' or status == 'в разводе' or status == 'вдовец / вдова':
        if children == 0:
            return 'один, без детей'
        if children != 0:
            return 'один, с детьми'
    
data['family_grouped'] = data.apply(married_or_not, axis=1)

We will create a pivot table for the family_grouped category, displaying:

* The total number of clients in each subcategory.
* The number of defaulters in each subcategory.

In [32]:
family_pivot_data = data.pivot_table(index='family_grouped', values='debt', aggfunc=['count', 'sum'])

family_pivot_data['debtors_percent'] = family_pivot_data['sum'] / family_pivot_data['count'] * 100
family_pivot_data['debtors_percent'] = family_pivot_data['debtors_percent'].round(decimals=2)

family_pivot_data.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
family_pivot_data.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
family_grouped,,,
в браке,12261,927,7.56
"один, без детей",3893,318,8.17
не в браке,4134,385,9.31
"один, с детьми",1043,102,9.78


As expected, people who have not formalized their relationships struggle more with loan repayments than those who are officially married. The least reliable borrowers are clients who have a dependent minor relative.

Let's examine this subcategory across other indicators—there might be some interesting patterns worth noting.

In [33]:
# We will create a pivot table by selecting single defaulters with children from the main dataset.

compare_family = data[(data['family_grouped'] == 'один, с детьми') & (data['debt'] == 1)].pivot_table(
    index=['total_income_category', 'family_status'], 
    values=['total_income', 'debt'], 
    aggfunc=['median', 'count']).reset_index() 


compare_family.columns = ['total_income_category', 'family_status', 'median_debt', 'median_total_income', 'debt_count', 'income_count']

compare_family.drop(['median_debt', 'income_count'], axis=1, inplace=True)

compare_family['ingroup_percent'] = compare_family['debt_count'] / 102 * 100

compare_family['ingroup_percent'] = compare_family['ingroup_percent'].round(decimals=2)

compare_family.sort_values('debt_count', ascending=False)

,total_income_category,family_status,median_total_income,debt_count,ingroup_percent
3,C,Не женат / не замужем,142594.0,51,50.00
4,C,в разводе,134714.0,21,20.59
0,B,Не женат / не замужем,242307.0,12,11.76
5,C,вдовец / вдова,148167.0,8,7.84
1,B,в разводе,223195.0,6,5.88
2,B,вдовец / вдова,303546.0,2,1.96
6,D,в разводе,45402.0,2,1.96


The table shows that half of the defaulters in this category have a median income of 142,594 RUB, which is close to the value we observed earlier when analyzing the relationship between the number of children and loan defaults.

**Conclusion**:
* The highest default rate (9.78%) is among single parents raising one or more children alone. Their median income is 134,714 RUB (divorced) and 142,594 RUB (never married).
* A slightly lower default rate (9.31%) is observed among clients in unregistered relationships. Likely, many take out loans while in a relationship but end up repaying them alone. However, our dataset lacks enough data for deeper analysis.
* Married couples with legally registered relationships have the lowest default rate (7.56%).Based on our previous analysis (Section 3.1), most married couples who successfully repay loans do not have children.

#### 3.3. Income Level vs. On-Time Loan Repayment
We will create a pivot table for the `total_income_category` column, displaying:

* The total number of clients in each income category.
* The number of defaulters in each income category.

In [34]:
income_category_pivot = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum'])

income_category_pivot['debtors_percent'] = income_category_pivot['sum'] / income_category_pivot['count'] * 100
income_category_pivot['debtors_percent'] = income_category_pivot['debtors_percent'].round(decimals=2)

income_category_pivot.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
income_category_pivot.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
total_income_category,,,
D,349,21,6.02
B,5014,354,7.06
A,25,2,8.00
C,15921,1353,8.50
E,22,2,9.09


Groups `A` (1M+ income) and `E` (up to 30K income) are too small to form a representative sample. It can be assumed that:

* Clients in category `A` rarely need loans in the first place.
* Clients in category `E` may face more frequent loan rejections.

However, our dataset does not provide enough information to verify these assumptions.  
Group `D` (30K–50K income) is also relatively small but has the lowest default rate, which is unusual given its income level.  
To investigate this, let's compare defaulters from categories B, C, and D—perhaps there are differences in loan purposes.

In [35]:
compare_income = data[((data['total_income_category'] == 'B')| (data['total_income_category'] == 'C') | (data['total_income_category'] == 'D')) & (data['debt'] == 1)].pivot_table(
    index=['total_income_category', 'purpose_category'],
    values=['total_income', 'debt'],
    aggfunc=['median', 'count']).reset_index() 

compare_income.columns = ['total_income_category', 'purpose_category', 'median_debt', 'median_total_income', 'debt_count', 'income_count']

compare_income.drop(['median_debt', 'income_count'], axis=1, inplace=True)

compare_income['ingroup_percent'] = compare_income.loc[compare_income['total_income_category'] == 'D', 'debt_count'] / 21 * 100
compare_income.loc[compare_income['total_income_category'] == 'B', 'ingroup_percent'] = compare_income.loc[compare_income['total_income_category'] == 'B', 'debt_count'] / 354 * 100
compare_income.loc[compare_income['total_income_category'] == 'C', 'ingroup_percent'] = compare_income.loc[compare_income['total_income_category'] == 'C', 'debt_count'] / 1353 * 100

compare_income['ingroup_percent'] = compare_income['ingroup_percent'].round(decimals=2)

compare_income.sort_values(['total_income_category','debt_count'], ascending=(True, False))

,total_income_category,purpose_category,median_total_income,debt_count,ingroup_percent
1,B,операции с недвижимостью,251859.0,172,48.59
0,B,операции с автомобилем,257949.0,85,24.01
2,B,получение образования,258652.0,69,19.49
3,B,проведение свадьбы,263052.0,28,7.91
5,C,операции с недвижимостью,134714.0,599,44.27
4,C,операции с автомобилем,134657.0,305,22.54
6,C,получение образования,128074.0,297,21.95
7,C,проведение свадьбы,123708.0,152,11.23
8,D,операции с автомобилем,45174.0,10,47.62
9,D,операции с недвижимостью,42610.0,6,28.57


Half of the defaulters in categories B and C took out loans related to real estate, while almost half of category D defaulters (the most reliable group) borrowed for car purchases. This suggests that auto loans might be the most problematic credit type, which we will analyze further in Section 3.4.

**Conclusion**:  
* The largest borrower group is category C (income 50,001–200,000 RUB), and they also have the highest default rate.
* Category D borrowers (income 30,001–50,000 RUB) show the best repayment reliability (6.02% default rate), but these findings need verification on a larger sample.
* Category B (income 200,001–1M RUB) also has a low default rate (7.06%), making them a relatively stable borrower group.  

*Important Note*:  
Previous analysis showed that family status and the presence of children significantly impact repayment ability. Therefore, income levels alone should not be considered in isolation when assessing creditworthiness.

#### 3.4. Loan Purpose vs. On-Time Loan Repayment 
We will create a pivot table for the purpose_category column, displaying:

* The total number of clients in each loan purpose category.
* The number of defaulters in each category.

In [36]:
purpose_pivot = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])

purpose_pivot ['debtors_percent'] = purpose_pivot ['sum'] / purpose_pivot ['count'] * 100
purpose_pivot ['debtors_percent'] = purpose_pivot ['debtors_percent'].round(decimals=2)

purpose_pivot.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
purpose_pivot.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
purpose_category,,,
операции с недвижимостью,10751,780,7.26
проведение свадьбы,2313,183,7.91
получение образования,3988,369,9.25
операции с автомобилем,4279,400,9.35


Earlier, we observed that within income group D, nearly half of the defaulters took out loans for car purchases. Now, analyzing defaults across the entire dataset confirms that "car-related loans" are the most problematic credit purpose.

**Conclusion:**  
The largest and most reliable borrower group consists of those who take out real estate loans, with a default rate of **7.26%**. Based on previous findings, these loans are more likely to be repaid on time by married individuals with an income of 50,001–200,000 RUB or 200,001–1,000,000 RUB.  
Car-related loans form the second-largest and least reliable borrower group, with a default rate of **9.35%**. Nearly 1 in 10 borrowers in this category fail to make timely payments.  

Before moving on to the final conclusions, let’s examine one more client parameter that might impact credit repayment ability: education level.

In [37]:
education_category_pivot = data.pivot_table(index=['education'], values='debt', aggfunc=['count', 'sum'])

education_category_pivot['debtors_percent'] = education_category_pivot['sum'] / education_category_pivot['count'] * 100
education_category_pivot['debtors_percent'] = education_category_pivot['debtors_percent'].round(decimals=2)

education_category_pivot.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
education_category_pivot.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
education,,,
ученая степень,6,0,0.00
высшее,5227,278,5.32
среднее,15075,1355,8.99
неоконченное высшее,741,68,9.18
начальное,282,31,10.99


For the two largest categories, higher education and secondary education, we can conclude that education level also influences a borrower's repayment ability.

### 4. General Conclusion

Each of the examined client parameters affects credit repayment ability:
* Children: Clients without children are less likely to have outstanding debt compared to those with two children (7.54% vs. 9.45%).
* Marital status: The lowest default rate (7.56%) is among married clients, while the highest (9.78%) is among single parents.
* Income: The risk of credit default decreases as income increases:
    * 7.06% of Group B (income 200,000–1,000,000 RUB) have debt.
    * 8.50% of Group C (income 50,000–200,000 RUB) have debt. However, Group B is three times smaller than Group C, making it necessary to collect more data to ensure a balanced comparison.
* Loan Purpose:
    * The highest default rate (9.35%) is among clients who take out loans for car purchases.
    * Real estate loans, despite their higher cost, have the lowest default rate (7.26%).
* Education: Higher education indirectly impacts creditworthiness:
    * Clients with higher education take out loans less often and have a lower default rate (5.32%) compared to those with secondary education (8.99%).


**Profile of the Most Reliable Borrower**  
A married client, without children, with higher education, and an income of 200,000–1,000,000 RUB.

**Profile of the Least Reliable Borrower**  
A single client, with 1–2 children, with secondary education, and an income of 50,000–200,000 RUB.

**Further Analysis Needed**  
For a more accurate assessment, all characteristics should be analyzed together to determine their correlation. For example:
* How education level, job position, and work experience influence income.
* The correlation between income, total household income, and loan repayment.
* The correlation between income, loan amount, and default rate.
This deeper analysis would help identify the most reliable borrowers more precisely.